In [ ]:
import ibis

In [ ]:
ibis.options.sql.default_limit = None

hdfs_conn = ibis.hdfs_connect(host='bottou03.sjc.cloudera.com')

ibis_conn = ibis.impala.connect(host='bottou01.sjc.cloudera.com',
                                    port=21050,
                                    hdfs_client=hdfs_conn)

In [ ]:
ibis_conn.drop_table('wiki_pageviews', database='u_juliet')

In [ ]:
hdfs_data_path = '/user/juliet'
hdfs_conn.mkdir(hdfs_data_path)

In [ ]:
hdfs_conn.ls('/user/juliet')

In [ ]:
import os
# System indepedent way to join paths
local_data_path = os.path.join(os.getcwd(), "pageviews-gz")

In [ ]:
filenames = os.listdir(local_data_path)
filenames[3][:-3]

must put each file in its own dir because we need info from file name, using impala.

In [ ]:
hdfs_conn.mkdir('/user/juliet/pageviews-gz')
def mv_files(filename):
    dir_name = '/user/juliet/pageviews-gz/{}'.format(filename[:-3])
    hdfs_conn.mkdir(dir_name)
    filepathtarget='/'.join([dir_name, filename])
    hdfs_conn.put(filepathtarget, os.path.join(local_data_path, filename))
    return dir_name
    
hdfs_gz_dirs = [mv_files(filename) for filename in filenames]

In [ ]:
hdfs_gz_dirs = hdfs_conn.ls('/user/juliet/pageviews-gz/')

In [ ]:
hdfs_conn.ls('/user/juliet')

In [ ]:
hdfs_gz_dirs = [ '/user/juliet/pageviews-gz/' + filename for filename in hdfs_conn.ls('/user/juliet/pageviews-gz')]
hdfs_gz_dirs

In [ ]:
import pandas as pd

def extract_datetime(filename):
    _, date_str, time_str = filename.split("-")
    year = date_str[:4]
    month = date_str[4:6]
    day = date_str[-2:]
    hour = time_str[:2]
    return year, month, day, hour

def to_pd_dt(filename):
    return pd.to_datetime(filename, format='pageviews-%Y%m%d-%H0000')

In [ ]:
# quick check that this does the right thing
extract_datetime('pageviews-20160102-030000.gz')
to_pd_dt('pageviews-20160102-030000')

Each file actually has important information about itself in its title. We need to decompressed each file (because gzip is not splittable and we could do better with compression codecs), read each into a dataset, concatenate all the datasets, write out the output in a better format. (Ie parquet with LZO compression)

We could do this with spark or impala. Spark writes parquiet files without row groups (perhaps not strictly incorrect, but impala does not handle this well) For best mutual compatibility we should write out our parquet file with Impala, not Spark.

# Making a better file
## Create Impala Table

In [ ]:
ibis_conn.create_database('u_juliet')

In [ ]:
file_schema = ibis.schema([('project_name', 'string'),
                           ('page_name', 'string'),
                           ('n_views', 'int64'),
                           ('n_bytes', 'int64')])

In [ ]:
pageviews_tbl = ibis_conn.table('wiki_pageviews', database='u_juliet')

In [ ]:
ibis_conn.execute(pageviews_tbl.count())

In [ ]:
u_juliet_db = ibis_conn.database('u_juliet')
u_juliet_db.tables

In [ ]:
ibis_conn.execute(pageviews_tbl.project_name.distinct())

In [ ]:
ibis_conn.execute(pageviews_tbl.limit(10))

In [ ]:
def gz_2_data_insert(data_dir, db_expr):
    tmp_table = ibis_conn.delimited_file(hdfs_dir=data_dir,
                                  schema=file_schema,
                                  delimiter=' ')
    year, month, day, hour = extract_datetime(data_dir.split("/")[-1])
    # create a column named time
    tmp_w_time = tmp_table.mutate(year=year, month=month, day=day, hour=hour)
    if 'wiki_pageviews' in db_expr.tables:
        ibis_conn.insert('wiki_pageviews', tmp_w_time, database='u_juliet')
    else:
        ibis_conn.create_table('wiki_pageviews', obj=tmp_w_time, database='u_juliet')

[gz_2_data_insert(data_dir, u_juliet_db) for data_dir in hdfs_gz_dirs]

In [ ]:
u_juliet_db.tables

In [ ]:
pageviews_tbl = ibis_conn.table('wiki_pageviews', database='u_juliet')

In [ ]:
[str(n).zfill(2) for n in range(1, 31)]